In [1]:
import pandas as pd
import numpy as np
import joblib
import re


In [2]:
df=pd.read_csv('C:/Users/Madhu/Desktop/ML_Project/forms.csv')
df.head()

,Timestamp,Username,Age,Which movie industry do you mostly watch?,Favorite actor,Favorite actor.1,what is your favorite movie,what is your favorite movie.1,what is your favorite movie.2,what is your favorite movie.3,...,What is your favorite movie,What is the highest ticket price you paid for a movie (even in black for one person),How many movie re-releases have you watched,What motivates you to go for a re-release?,which do you enjoy more ?,How often do you go to theatres,Do you usually watch first day first show of re-release,How excited do you feel when an old blockbuster is re-released ?,Would you go again for another re-release in the future?,"If you got a chance to watch one re-released movie again, which one would you choose?"
0,2025/11/10 11:00:36 pm GMT+5:30,sankabaktulamoksha3soham12@gmail.com,20,Telugu,Prabhas,NaN,NaN,NaN,NaN,Baahubali,...,NaN,300,0,Actor,Both,Often,Yes,5,Yes,Salaar
1,2025/11/10 11:02:12 pm GMT+5:30,onepiece2746@gmail.com,20,Telugu,Prabhas,NaN,NaN,NaN,NaN,Kalki,...,NaN,150,0,Mass Scenes,Normal movie releases,Rarely,No,1,No,NaN
2,2025/11/10 11:04:06 pm GMT+5:30,ayubnaz710@gmail.com,19,Telugu,Prabhas,NaN,NaN,NaN,NaN,Baahubali,...,NaN,300,0,Nostalgia,Normal movie releases,Rarely,No,5,No,NaN
3,2025/11/10 11:08:12 pm GMT+5:30,durgeswarpeddinti@gmail.com,20,Telugu,Prabhas,NaN,NaN,NaN,NaN,Salaar,...,NaN,300,3,Mass Scenes,Both,Occasionally,Yes,5,Yes,Salaar
4,2025/11/10 11:09:58 pm GMT+5:30,pachamadhu1@gmail.com,20,Telugu,Jr. NTR,NaN,NaN,Temper,NaN,NaN,...,NaN,300,3,Actor,Both,Occasionally,No,4,Yes,Devera


In [3]:
# Combine favourite movie columns
df['fav_mov'] = df[[
    'what is your favorite movie','what is your favorite movie.1',
    'what is your favorite movie.2','what is your favorite movie.3',
    'what is your favorite movie.4','what is your favorite movie.5',
    'what is your favorite movie.6','what is your favorite movie.7',
    'what is your favorite movie.8','what is your favorite movie.9',
    'what is your favorite movie.10','what is you favorite movie',
    'what is your favorite movie.11','What is your favorite movie'
]].bfill(axis=1).iloc[:, 0]

In [4]:
# Combine favourite actor
df['fav_actor'] = df[['Favorite actor','Favorite actor.1']].bfill(axis=1).iloc[:, 0]

# Drop useless columns
df.drop(columns=[
    'Favorite actor','Favorite actor.1','Timestamp','Username',
    'what is your favorite movie','what is your favorite movie.1',
    'what is your favorite movie.2','what is your favorite movie.3',
    'what is your favorite movie.4','what is your favorite movie.5',
    'what is your favorite movie.6','what is your favorite movie.7',
    'what is your favorite movie.8','what is your favorite movie.9',
    'what is your favorite movie.10','what is you favorite movie',
    'what is your favorite movie.11','What is your favorite movie'
], inplace=True, errors="ignore")


In [5]:
# Rename columns
df.rename(columns={
    'Which movie industry do you mostly watch?':'Language',
    'What is the highest ticket price you paid for a movie (even in black for one person)':'Ticket_price',
    'How many movie re-releases have you watched':'Watch_count',
    'What motivates you to go for a re-release?':'Interest',
    'which do you enjoy more ?':'Movie_pref',
    'How often do you go to theatres':'Theatre_count',
    'Do you usually watch first day first show of re-release':'fdfs',
    'How excited do you feel when an old blockbuster is re-released ?':'Old_movies',
    'Would you go again for another re-release in the future?  ':'Go_again',
    'If you got a chance to watch one re-released movie again, which one would you choose? ':'Movie_choice'
}, inplace=True)


In [6]:
print(df.head())
df.columns

   Age Language Ticket_price Watch_count     Interest             Movie_pref  \
0   20   Telugu          300           0        Actor                   Both   
1   20   Telugu          150           0  Mass Scenes  Normal movie releases   
2   19   Telugu          300           0    Nostalgia  Normal movie releases   
3   20   Telugu          300           3  Mass Scenes                   Both   
4   20   Telugu          300           3        Actor                   Both   

  Theatre_count fdfs  Old_movies Go_again Movie_choice    fav_mov fav_actor  
0         Often  Yes           5      Yes       Salaar  Baahubali   Prabhas  
1        Rarely   No           1       No          NaN      Kalki   Prabhas  
2        Rarely   No           5       No          NaN  Baahubali   Prabhas  
3  Occasionally  Yes           5      Yes       Salaar     Salaar   Prabhas  
4  Occasionally   No           4      Yes       Devera     Temper   Jr. NTR  


Index(['Age', 'Language', 'Ticket_price', 'Watch_count', 'Interest',
       'Movie_pref', 'Theatre_count', 'fdfs', 'Old_movies', 'Go_again',
       'Movie_choice', 'fav_mov', 'fav_actor'],
      dtype='object')

### Cleaning data

In [7]:
def clean_text(text):
    # """Clean text data and replace invalid entries with 'salaar'"""
    
    # if pd.isna(text) or text is None:
    #     return 'salaar'
    
    text = str(text).strip().lower()
    text = re.sub(r'\s+', ' ', text)
    
    invalid_entries = [
        'None','none', 'no', 'na', 'n/a', '.', '-', 'nothing', 
        'all', 'everyone', 'any', 'idk', 'don\'t know', 'not sure',
        'no favorite', 'no favourite', 'no one', 'all movies', 
        'all movies are my favourite', 'it\'s subjective', 'subjective',
        'tfi banisa', 'whloe tfi', 'whole tfi', 'all heros', 
        'i am all heros fan', 'no favourite i just watch everyone\'s movies',
        'based on movie', 'yes', 'nothing', 'all ', 'any pawan kalyan movie',
        'awaara, salaar and many more', 'saalar and bahubali', 
        'awaara, orange, salaar and many more', 'salaar,chakram','Yes','yes'
    ]
    
    if text in invalid_entries or text == '' or len(text) <= 2:
        return 'salaar'
    
    if ',' in text or ' and ' in text or 'many more' in text:
        return 'salaar'
    
    return text


In [8]:
def standardize_actor_name(actor):
    # """Standardize actor name variations"""
    if not actor or actor == 'salaar':
        return 'prabhas'
    
    actor = str(actor).strip().lower()
    
    actor_map = {
        'prabhas': ['prabhas', 'prabhass', 'prabas', 'rebel star prabhas','All','all',
                    'No favourite I just watch everyones movies','Whloe TFI','I am all heros fan',
                    'TFI banisa','No one ','No favorite','None','no one',''],
        'ram charan': ['ram charan', 'ramcharan', 'rc', 'cherry'],
        'jr. ntr': ['jr ntr', 'jr.ntr', 'ntr jr', 'tarak', 'young tiger', 'jr. ntr'],
        'allu arjun': ['allu arjun', 'alluarjun', 'aa', 'bunny', 'stylish star'],
        'mahesh babu': ['mahesh babu', 'maheshbabu', 'mahesh', 'superstar mahesh'],
        'pawan kalyan': ['pawan kalyan', 'pawankalyan', 'pk', 'power star'],
        'vijay deverakonda': ['vijay devarakonda', 'vijay deverakonda', 'vijay devarakonda ', 'vd'],
        'nani': ['nani', 'natural star nani'],
        'ravi teja': ['ravi teja', 'raviteja', 'mass maharaja'],
        'balakrishna': ['balaiah', 'balakrishna', 'nandamuri balakrishna', 'nbk'],
        'yash': ['yash', 'rocking star yash'],
        'bellamkonda srinivas': ['bellamkonda srinivas', 'bellamkonda'],
        'raghava lawrence': ['raghava lawrance', 'raghava lawrence', 'lawrence'],
        
        # Hindi actors
        'shah rukh khan': ['shah rukh khan', 'shahrukh khan', 'srk', 'king khan'],
        'salman khan': ['salman khan', 'salman', 'bhai'],
        'aamir khan': ['aamir khan', 'amir khan', 'aamir'],
        'hrithik roshan': ['hrithik roshan', 'hrithik', 'duggu'],
        'ranbir kapoor': ['ranbir kapoor', 'ranbir', 'rk'],
        'akshay kumar': ['akshay kumar', 'akshay', 'khiladi'],
        'sushant singh rajput': ['sushant singh rajput', 'sushant', 'ssr'],
        'varun dhawan': ['varun dhawan', 'varun dhawan ', 'varun'],
        'siddharth malhotra': ['siddharth malhotra', 'siddharth malhotra ', 'sidharth'],
        'irrfan khan': ['irfan khan', 'irrfan khan', 'irrfan'],
        
        # Tamil actors
        'vijay': ['vijay talapathy', 'vijay thalapathy', 'vijay', 'thalapathy'],
        
        # Other
        'r. madhavan': ['r. madhavan', 'madhavan', 'maddy'],
        'dulquer salmaan': ['dulquer salmaan', 'dulquer salmaan ', 'dq'],
        'lakshya': ['lakshya'],
        'kay kay menon': ['kk menon', 'kay kay menon'],
        'kareena kapoor': ['kareena kapoor', 'kareena kapoor ', 'bebo']
    }
    
    for standard_name, variations in actor_map.items():
        if actor in variations:
            return standard_name
    
    return actor


In [9]:
def standardize_movie_name(movie):
    # """Standardize movie name variations"""
    if not movie:
        return 'salaar'
    
    movie = movie.strip().lower()
    movie = movie.replace('(telugu)', '').replace('()', '').strip()
    
    movie_map = {
        'baahubali': ['baahubali', 'bahubali', 'bahubali 1', 'bahubali 2', 'baahubali 1', 
                      'baahubali 2', 'bhahubali the epic', 'bahubali '],
        'rrr': ['rrr', 'r r r', 'rrrr','Rrr','RRR'],
        'pushpa': ['pushpa', 'pushpa 1', 'pushpa part 1'],
        'salaar': ['salaar', 'salar', 'salaar part 1', 'saalaar', 'salaar '],
        'kalki': ['kalki', 'kalki 2898 ad', 'kalki 2898', 'kalki, kgf'],
        'magadheera': ['magadheera', 'magadeera', 'magahdeera'],
        'rangasthalam': ['rangasthalam', 'rangastalam'],
        'pokiri': ['pokiri', 'pokkiri','Ssmb29','ssmb29'],
        'mirchi': ['mirchi', 'mirapakay', 'mirchi (telugu)'],
        'businessman': ['businessman', 'business man', 'buisnessman', 'businessman '],
        'khaleja': ['khaleja', 'khaleeja', 'khaleja '],
        'gabbar singh': ['gabbar singh', 'gabbar', 'gabbarsingh'],
        'attarintiki daredi': ['attarintiki daredi', 'attarintiki daaredi', 'ad'],
        'simhadri': ['simhadri', 'simahadri', 'shimhadri'],
        'temper': ['temper', 'temperr'],
        'saaho': ['saaho', 'sahooo'],
        'darling': ['darling', 'darling movie', 'darling '],
        'orange': ['orange', 'orange '],
        'jersey': ['jersey', 'jersey '],
        'color photo': ['color photo', 'color photo'],
        'arjun reddy': ['arjun reddy', 'arjun reddy '],
        'bhramotsavam': ['bhramotsavam', 'brahmotsavam'],
        'johnny': ['johnny', 'johnny '],
        'badri': ['badri', 'badri '],
        'chatrapati': ['chatrapati', 'chatrapati '],
        'happy days': ['hapoy days', 'happy days'],
        'sarrainodu': ['sarrinodu', 'sarrainodu', 'sarrinodu '],
        'oy': ['oy', 'oy!', 'oy! '],
        'one': ['one','1 nenokkadine'],
        'ala vaikunthapurramuloo': ['ala vaikuntapuram lo', 'ala vaikunthapurramuloo'],
        'raja varu rani garu': ['raja varu rani garu', 'raja varu rani garu '],
        'geetha govindam': ['gita govindam', 'geetha govindam'],
        'janatha garage': ['janatha garage'],
        'aravinda sametha': ['aravinda sametha', 'aravinda sametha '],
        'gang leader': ['gang leader', 'gang leader '],
        'panja': ['panja', 'panja '],
        'maryadha ramanna': ['maryadharaamanna'],
        '1 nenokkadine': ['1 nenokkadine'],
        
        '3 idiots': ['3 idiots', '3idiots', 'three idiots', '3 idiots '],
        'ddlj': ['ddlj', 'dilwale dulhania le jayenge'],
        'pathaan': ['pathaan', 'pathan'],
        'dil bechara': ['dil bechara', 'dil bechara', 'dilbechara', 'dil bechare', 
                        'dil bechara ', 'dil bechare '],
        'chhichhore': ['chhichhore', 'chichhore', 'chhichore', 'chhichhore '],
        'yeh jawaani hai deewani': ['yeh jwani hai deewani', 'ye jawaani hai dewaani', 
                                    'yeh jawaani hai deewani'],
        'kalank': ['kalank', 'kalank '],
        'sanam teri kasam': ['sanam teri kasam', 'sanam teri kasam-movie'],
        
        'spider-man': ['spiderman', 'spiderman ', 'spider-man no way home'],
        'avengers endgame': ['avengers:endgame', 'avengers endgame'],
        
        'avane srimannarayana': ['avane srimannarayana', 'avane srimannarayana'],
        'gaalipata': ['gaalipata', 'gaalipata '],
        'tumbbad': ['thumbad', 'tumbbad'],
        'hi nanna': ['hi nanna', 'hi nanna'],
        'adbhutham': ['adbutham', 'adbhutham'],
        'barfi': ['barfi', 'barfi!'],
        'style': ['style', 'style '],
        'jaya janaki nayaka': ['jaya janaki nayaka'],
        'lucky bhaskar': ['lucky bhaskar', 'lucky bhaskar '],
        'murari': ['murari', 'murari '],
        'inglourious basterds': ['inglorious bastard', 'inglourious basterds'],
        'kGF': ['k3g','KGF','Kgf','kgf','KGF 2','Kgf 2','kgf 2'],

    }
    
    for standard_name, variations in movie_map.items():
        if movie in variations:
            return standard_name
    
    return movie


#### Applying Cleaning


In [10]:
df['fav_actor'] = df['fav_actor'].apply(clean_text).apply(standardize_actor_name)
df['fav_mov'] = df['fav_mov'].apply(clean_text).apply(standardize_movie_name)
df['Movie_choice'] = df['Movie_choice'].apply(clean_text).apply(standardize_movie_name)


## Data Preprocessing

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Age            208 non-null    int64 
 1   Language       208 non-null    object
 2   Ticket_price   208 non-null    object
 3   Watch_count    208 non-null    object
 4   Interest       208 non-null    object
 5   Movie_pref     208 non-null    object
 6   Theatre_count  208 non-null    object
 7   fdfs           208 non-null    object
 8   Old_movies     208 non-null    int64 
 9   Go_again       208 non-null    object
 10  Movie_choice   208 non-null    object
 11  fav_mov        208 non-null    object
 12  fav_actor      208 non-null    object
dtypes: int64(2), object(11)
memory usage: 21.3+ KB


In [12]:
df.describe()
# df['Movie_choice'].head()

,Age,Old_movies
count,208.000000,208.000000
mean,19.975962,3.634615
std,3.548749,1.380035
min,13.000000,1.000000
25%,19.000000,3.000000
50%,20.000000,4.000000
75%,21.000000,5.000000
max,65.000000,5.000000


In [13]:
df.columns

Index(['Age', 'Language', 'Ticket_price', 'Watch_count', 'Interest',
       'Movie_pref', 'Theatre_count', 'fdfs', 'Old_movies', 'Go_again',
       'Movie_choice', 'fav_mov', 'fav_actor'],
      dtype='object')

In [14]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
le =LabelEncoder()

a=['Language', 'Ticket_price', 'Watch_count', 'Interest',
       'Movie_pref', 'Theatre_count', 'fdfs', 'Old_movies', 'Go_again',
       'Movie_choice', 'fav_mov', 'fav_actor']

for col in a:
    df[col+"_le"]=le.fit_transform(df[col])




## Goal 1: Classification

Predicting wheather user is intrested in watching re-release or not

In [16]:
df.columns

Index(['Age', 'Language', 'Ticket_price', 'Watch_count', 'Interest',
       'Movie_pref', 'Theatre_count', 'fdfs', 'Old_movies', 'Go_again',
       'Movie_choice', 'fav_mov', 'fav_actor', 'Language_le',
       'Ticket_price_le', 'Watch_count_le', 'Interest_le', 'Movie_pref_le',
       'Theatre_count_le', 'fdfs_le', 'Old_movies_le', 'Go_again_le',
       'Movie_choice_le', 'fav_mov_le', 'fav_actor_le'],
      dtype='object')

In [17]:
features=['Language_le',
       'Ticket_price_le', 'Watch_count_le', 'Interest_le', 'Movie_pref_le',
       'Theatre_count_le', 'fdfs_le', 'Old_movies_le', 'fav_mov_le', 'fav_actor_le']

x=df[features]
y=df['Go_again_le']

st_scaler=StandardScaler()

x1_scaled=st_scaler.fit_transform(x)

x_train,x_test,y_train,y_test=train_test_split(x1_scaled,y,test_size=0.2,random_state=42)


### MODEL 1: CLASSIFICATION

### Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,r2_score,mean_squared_error,mean_absolute_error, classification_report, confusion_matrix

logistic=LogisticRegression()

logistic.fit(x_train,y_train)

y_pred_logi=logistic.predict(x_test)

print("Accuracy score: ",accuracy_score(y_test,y_pred_logi))
print("Classification Report: \n", classification_report(y_test, y_pred_logi))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred_logi))

Accuracy score:  0.7857142857142857
Classification Report: 
               precision    recall  f1-score   support

           0       0.89      0.50      0.64        16
           1       0.76      0.96      0.85        26

    accuracy                           0.79        42
   macro avg       0.82      0.73      0.74        42
weighted avg       0.81      0.79      0.77        42

Confusion Matrix: 
 [[ 8  8]
 [ 1 25]]


### KNearest Kneibhour

In [19]:
from sklearn.neighbors import KNeighborsClassifier

knn=KNeighborsClassifier(n_neighbors=19)
knn.fit(x_train,y_train)

y_pred_knn=knn.predict(x_test)

print("Accuracy score: ",accuracy_score(y_test,y_pred_knn))
print("Classification Report: \n", classification_report(y_test, y_pred_knn))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred_knn))

Accuracy score:  0.7380952380952381
Classification Report: 
               precision    recall  f1-score   support

           0       0.78      0.44      0.56        16
           1       0.73      0.92      0.81        26

    accuracy                           0.74        42
   macro avg       0.75      0.68      0.69        42
weighted avg       0.75      0.74      0.72        42

Confusion Matrix: 
 [[ 7  9]
 [ 2 24]]


## Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier,plot_tree

Dtree = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=4,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

Dtree.fit(x_train,y_train)

y_pred_Dtree=Dtree.predict(x_test)

print("Accuracy score: ",accuracy_score(y_test,y_pred_Dtree))
print("Classification Report: \n", classification_report(y_test, y_pred_Dtree))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred_Dtree))

Accuracy score:  0.7142857142857143
Classification Report: 
               precision    recall  f1-score   support

           0       0.70      0.44      0.54        16
           1       0.72      0.88      0.79        26

    accuracy                           0.71        42
   macro avg       0.71      0.66      0.67        42
weighted avg       0.71      0.71      0.70        42

Confusion Matrix: 
 [[ 7  9]
 [ 3 23]]


In this Clasification models 
1. Logistic Regression has an accuracy of 78%
2. KNN Has 73%
3. Decision Tree has 71% 


with this now i'm using " Logistic Regression " as the main model to Predict 
Wheather User is intrested in watching Re-Release or  not

In [21]:
df['Movie_choice'].head()

0    salaar
1       nan
2       nan
3    salaar
4    devera
Name: Movie_choice, dtype: object

# Goal 2 : Movie Recomandation

if user intrested in watching re-release's then which movie user might intrested in 

### MODEL 2: CONTENT-BASED FILTERING

In [22]:
# Filter only rows where people said "Yes"
df_yes = df[df['Go_again'] == 'Yes'].copy()

# Keep only required columns
df_movie = df_yes[[ 'Language', 'Interest','Movie_pref', 'fdfs','Movie_choice', 'fav_mov', 'fav_actor']].copy()

# Remove duplicate movies (keep first entry)
df_movie = df_movie.drop_duplicates(subset=['Movie_choice']).reset_index(drop=True)

# Create a simple text content column
df_movie['content'] = (
    df_movie['Movie_choice'] + " " +
    df_movie['fav_actor'] + " " +
    df_movie['fav_mov'] + " " +
    df_movie['Language'] + " " +
    df_movie['Interest']
)


df_movie.head()

,Language,Interest,Movie_pref,fdfs,Movie_choice,fav_mov,fav_actor,content
0,Telugu,Actor,Both,Yes,salaar,baahubali,prabhas,salaar prabhas baahubali Telugu Actor
1,Telugu,Actor,Both,No,devera,temper,jr. ntr,devera jr. ntr temper Telugu Actor
2,Telugu,Mass Scenes,Both,Yes,arya,pushpa,allu arjun,arya allu arjun pushpa Telugu Mass Scenes
3,Hindi,Nostalgia,Normal movie releases,No,sultan,bajrangi bhaijaan,salman khan,sultan salman khan bajrangi bhaijaan Hindi Nos...
4,Telugu,Nostalgia,Both,Yes,panja,salaar,pawan kalyan,panja pawan kalyan salaar Telugu Nostalgia


In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english', max_features=100)
tfidf_matrix = tfidf.fit_transform(df_movie['content'])

# Cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix)

# Create index maps
movie_to_idx = {movie: idx for idx, movie in enumerate(df_movie['Movie_choice'])}
idx_to_movie = {idx: movie for movie, idx in movie_to_idx.items()}

print("Similarity matrix shape:", cosine_sim.shape)

Similarity matrix shape: (80, 80)


In [25]:
joblib.dump(logistic, "logistic.joblib")
joblib.dump(le, "label_encoders.joblib")
joblib.dump(st_scaler, "scaler_interest.joblib")

content_based_model = {
    'movie_database': df_movie,
    'tfidf': tfidf,
    'tfidf_matrix': tfidf_matrix,
    'cosine_sim': cosine_sim,
    'movie_to_idx': movie_to_idx,
    'idx_to_movie': idx_to_movie
}
joblib.dump(content_based_model, "Content_based_recomendation.joblib")

cleaning_functions = {
    'clean_text': clean_text,
    'standardize_actor_name': standardize_actor_name,
    'standardize_movie_name': standardize_movie_name
}
joblib.dump(cleaning_functions, "cleaning_functions.joblib")


['cleaning_functions.joblib']